In [1]:
#irr
import mxnet 
from mxnet.gluon import nn
from mxnet import gluon, image, np, npx
import matplotlib.pyplot as plt
import numpy
from mxnet.gluon.data.vision import transforms
import cv2
from PIL import Image
from mxnet.gluon.model_zoo import vision
import os
import pandas as pd
from d2l import mxnet as d2l
from mxnet.gluon import nn
from mxnet import init
npx.set_np()

In [2]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    return npx.gpu(i) if npx.num_gpus() >= i + 1 else npx.cpu()

def try_all_gpus():
    """Return all available GPUs, or [cpu()] if no GPU exists."""
    devices = [npx.gpu(i) for i in range(npx.num_gpus())]
    return devices if devices else [npx.cpu()]


def read_dataset():
    """read data from file pre known and returns image array, unique image add,name and dict of 
    {name:bbox list}."""
    data = pd.read_csv('json_to_csv_with_address.csv')
    x=data.loc[:,['image_address','image_name']]
    x=x.drop_duplicates()
    img_real = [cv2.imread(item) for item in x['image_address']]
    img_real = [cv2.cvtColor(item, cv2.COLOR_BGR2RGB) for item in img_real]
    
    y = data.loc[:,['ymin','xmin','ymax','xmax','label']]
    boxes = dict({item:[] for item in x['image_name']})
    
    for i,name in enumerate(data['image_name']):
        boxes[name].append([y.loc[i]])

    
    return img_real,x, boxes

def return_bbox_with_label(x,name):
    '''takes input x=dict of {name: bbox}, name= list of unique image names .
    and returns bbox , label as list'''
    temp = x[name]
    lst = []
    label = []
    for i in temp:
        for j in i:
            lst.append([j[0],j[1],j[2],j[3]])
            label.append(j[4])
    return lst,label
    
def plot_im_box(im,box,label=None):
    '''plot boxes on image im and returns plot fig'''
    image = im.copy()
    color = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(255,0,255),(0,255,255)]
    for j,i in enumerate(box):
        thickness= 2

        image = cv2.rectangle(image, (int(i[1]),int(i[0])) ,(int(i[3]),int(i[2])), color[j%len(color)], thickness)
        if label!=None:
            cv2.putText(image, text= label[j], org=(int(i[1]),int(i[0])),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, color=(255,255,255),
                    thickness=2, lineType=cv2.LINE_AA)
    #cv2.imshow(window_name, image)
    return plt.imshow(image)

def box_iou(boxes1, boxes2):
    """Compute IOU between two sets of boxes of shape (N,4) and (M,4)."""
    # Compute box areas
    box_area = lambda boxes: ((boxes[:, 2] - boxes[:, 0]) *
                              (boxes[:, 3] - boxes[:, 1]))
    area1 = box_area(boxes1)
    area2 = box_area(boxes2)
    lt = np.maximum(boxes1[:, None, :2], boxes2[:, :2])  # [N,M,2]
    rb = np.minimum(boxes1[:, None, 2:], boxes2[:, 2:])  # [N,M,2]
    wh = (rb - lt).clip(min=0)  # [N,M,2]
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]
    unioun = area1[:, None] + area2 - inter
    return inter / unioun

def cross_entropy(y_hat, y):
    ce = -np.log(y_hat[range(len(y_hat)), y])
    return ce.mean()


In [3]:
device = try_gpu()
ctx = npx.cpu()
all_device = try_all_gpus()

data = pd.read_csv('json_to_csv_with_address.csv')
z = data['label'].drop_duplicates()
label_id = dict({item:i for i,item in enumerate(z)})
label_name = []
for item in z:
    label_name.append(item)
num_labels = len(label_name)

im,x, b = read_dataset()     #im for image used in cv2.rect, b for dictionary of {image_name:[bbox,label]}, x for df of [locn,im_name]
img = np.array(im, dtype= np.float32) #dummy of im

model = vision.vgg16(pretrained=True )    #pretrained vgg16 model 
fe = list(model.features)


dummy_img = np.zeros((1,3,800,800), dtype = np.float32)

k= np.array(dummy_img , dtype= np.float32)
req_features=[]
for i in fe:
    k= i(k)
    if k.shape[2]<800//16:
        break
    req_features.append(i)
    out_channels = k.shape[1]


#after res some irr 
ratios= [0.5,1,2]
anchor_scales=[8,16,32]
sub_sample = 16 
ctr_y = sub_sample/ 2.
ctr_x = sub_sample/ 2.

fe_size = 800//16
sub_sample = 16 
ctr_x = np.arange(sub_sample, (fe_size+1)*sub_sample, sub_sample)
ctr_y = np.arange(sub_sample, (fe_size+1)*sub_sample, sub_sample)

ctr = np.empty((len(ctr_x)*len(ctr_y),2), dtype = np.int32)
index = 0 
for xx in range(len(ctr_x)):
    for yy in range(len(ctr_y)):
        ctr[index,1]= ctr_x[xx] - 8
        ctr[index,0]= ctr_y[yy] - 8
        index +=1

        
index = 0 
anchors = np.zeros(((fe_size*fe_size*9),4))
for c in ctr:
    ctr_y, ctr_x = c
    for i in range(len(ratios)):
        for j in range(len(anchor_scales)):
            h = sub_sample * anchor_scales[j]* np.sqrt(ratios[i])
            w = sub_sample * anchor_scales[j]* np.sqrt(1./ratios[i])
            
            anchors[index, 0] = ctr_y - h/2
            anchors[index, 1] = ctr_x - w/2
            anchors[index, 2] = ctr_y + h/2
            anchors[index, 3] = ctr_x + w/2
            index +=1
            
anchor_boxes = anchors
index_inside = np.where(
        (anchor_boxes[:, 0] >= 0) &
        (anchor_boxes[:, 1] >= 0) &
        (anchor_boxes[:, 2] <= 800) &
        (anchor_boxes[:, 3] <= 800)
    )[0]
valid_anchor_boxes = anchor_boxes[index_inside]

anc_height = anchor_boxes[:, 2] - anchor_boxes[:, 0]
anc_width = anchor_boxes[:, 3] - anchor_boxes[:, 1]
anc_ctr_y = anchor_boxes[:, 0] + 0.5 * anc_height
anc_ctr_x = anchor_boxes[:, 1] + 0.5 * anc_width


mid_channels = 512
in_channels = 512
n_anchor = 9

conv1 = nn.Conv2D(mid_channels, kernel_size=3, strides=1, padding=1,in_channels=in_channels)
reg_layer = nn.Conv2D(n_anchor*4, kernel_size=1, strides = 1, padding=0,activation = 'relu',in_channels=mid_channels)
cls_layer = nn.Conv2D(n_anchor*2, kernel_size=1, strides = 1, padding= 0,activation = 'softrelu',in_channels=mid_channels)

#conv1
conv1.weight.initialize(init.Normal(sigma=0.01))
#reg_layer
reg_layer.weight.initialize(init.Normal(sigma=0.01))
#cls_layer
cls_layer.weight.initialize(init.Normal(sigma=0.01))


conv1.bias.initialize(init.Zero())
reg_layer.bias.initialize(init.Zero())
cls_layer.bias.initialize(init.Zero())

nms_thresh = 0.7  # non-maximum supression (NMS) 
n_train_pre_nms = 12000 # no. of train pre-NMS
n_train_post_nms = 2000 # after nms, training Fast R-CNN using 2000 RPN proposals
n_test_pre_nms = 6000
n_test_post_nms = 300 # During testing we evaluate 300 proposals,
min_size = 16

n_sample = 128  # Number of samples from roi 
pos_ratio = 0.25 # Number of positive examples out of the n_samples
pos_iou_thresh = 0.5  # Min iou of region proposal with any groundtruth object to consider it as positive label
neg_iou_thresh_hi = 0.5  # iou 0~0.5 is considered as negative (0, background)
neg_iou_thresh_lo = 0.0

#final classification
roi_head_classifier = nn.Sequential()
roi_head_classifier.add(nn.Dense(4096, activation='relu'),
                       nn.Dense(4096, activation = 'relu'))
roi_head_classifier.collect_params().initialize(init.Normal(sigma=0.01))

cls_loc = nn.Dense(10*4, activation= 'relu')
cls_loc.weight.initialize(init.Normal(sigma=0.01))
cls_loc.bias.initialize(init.Zero())
score_l= nn.Dense(10, activation = 'softrelu')
score_l.weight.initialize(init.Normal(sigma=0.01))
score_l.bias.initialize(init.Zero())

In [4]:
j=0  #it can be 0 to 130 only  box = list of bbox, l = label of bbox
box,l = return_bbox_with_label(b,x['image_name'][x.index[j]])
bbox = np.array(box, dtype= np.float32)
label_l = l   #list of label names
label_s = [label_id[item] for item in l]   #list of label id's
labels = np.array(label_s, dtype= np.int32)  #np array of label id's
      #1 bit in feature map of out_channels corresponds to 16*16 of raw

img0= img[j]
transform = transforms.Compose([transforms.ToTensor()])
#imgTensor= transform(img0)
imgTensor = np.expand_dims(transform(img0),axis=0)
net = nn.Sequential()
with net.name_scope():
    for x in req_features:
        net.add(x)

out_map=net(imgTensor)
out_map.shape

ious = np.zeros((len(valid_anchor_boxes), len(bbox)), dtype=np.float32)
ious = box_iou(valid_anchor_boxes,bbox)
gt_argmax_ious_row = ious.argmax(axis=0)
gt_max_ious = ious[gt_argmax_ious_row, np.arange(ious.shape[1])]
gt_argmax_ious = np.where(ious == gt_max_ious)[0]
argmax_ious = ious.argmax(axis=1)
max_ious = ious[np.arange(len(index_inside)), argmax_ious]

label = np.zeros((len(index_inside), ), dtype=np.int32)-1

pos_iou_threshold  = 0.7
neg_iou_threshold = 0.3
label[gt_argmax_ious] = 1
label[max_ious >= pos_iou_threshold] = 1
label[max_ious < neg_iou_threshold] = 0

n_sample = 256
pos_ratio = .5
n_pos = int(pos_ratio * n_sample)

pos_index = np.where(label == 1)[0]

if len(pos_index)>n_pos:
    disable_index = np.random.choice(pos_index, 
                                     size = (len(pos_index) - n_pos), 
                                     replace = False)
    label[disable_index] = -1
    
    
n_neg =int( n_sample * np.sum(label == 1))
neg_index =np.where(label==0)[0]
#neg_index = np.array(np.where(label == 0), dtype= np.int64).reshape(-1,1)
if len(neg_index)> n_neg:
    disable_index = np.random.choice(neg_index, size= (len(neg_index) - (n_neg)), replace = False)
    label[disable_index]=-1

max_iou_bbox = bbox[argmax_ious]      # associativity of anchor to its corresponding ground truth bbox
#print(max_iou_bbox.shape)

height = valid_anchor_boxes[:,2] - valid_anchor_boxes[:,0]
width = valid_anchor_boxes[:,3] - valid_anchor_boxes[:,1]
ctr_y = valid_anchor_boxes[:,0]+ 0.5*height
ctr_y = valid_anchor_boxes[:,1]+ 0.5*width

base_height = max_iou_bbox[:,2] - max_iou_bbox[:,0]
base_width = max_iou_bbox[:,3] - max_iou_bbox[:,1]
base_ctr_y = max_iou_bbox[:,0] + 0.5*base_height
base_ctr_x = max_iou_bbox[:,1] + 0.5*base_width

eps = np.finfo(height.dtype).eps

height = np.maximum(height, eps)  #done to remove devide by zero error
width = np.maximum(width, eps)

dy = (base_ctr_y - ctr_y)/ height
dx = (base_ctr_x - ctr_x)/ width
dh = np.log(base_height/ height)
dw = np.log(base_width/ width)

anchor_locs = np.vstack((dy,dx,dh,dw))

#print(anchor_locs)
anchor_locs = anchor_locs.transpose()
#print(anchor_locs.shape, anchor_locs)

anchor_labels = np.zeros((len(anchors),), dtype = label.dtype)-1
anchor_labels[index_inside] = label

anchor_locations = np.zeros((len(anchors),anchors.shape[1]), dtype = anchor_locs.dtype)
anchor_locations[index_inside,:] = anchor_locs

z= conv1(out_map)
pred_anchor_locs = reg_layer(z)
pred_cls_scores = cls_layer(z)


pred_anchor_locs = np.transpose(pred_anchor_locs, (0,2,3,1)). reshape(1,-1,4)
pred_cls_scores = np.transpose(pred_cls_scores,(0,2,3,1))
objectness_score= pred_cls_scores.reshape(1,50,50,9,2)[:,:,:,:,1].reshape(1,-1)
pred_cls_scores= pred_cls_scores.reshape(1,50,50,9,2).reshape(1,-1,2)

rpn_loc = pred_anchor_locs[0]
rpn_score = pred_cls_scores[0]
gt_rpn_loc = anchor_locations
gt_rpn_score = anchor_labels

#loss start
##rpn_cls_loss = cross_entropy( rpn_score,gt_rpn_score)
ce_los = gluon.loss.SoftmaxCrossEntropyLoss()
rpn_cls_loss= ce_los(rpn_score,gt_rpn_score).mean()
#loss over

pos = (gt_rpn_score>0 ).reshape(-1,1)
mask = np.ones((pos.shape[1],4), dtype= pos.dtype)
mask = pos * mask

mask_loc_preds = rpn_loc[mask].reshape(-1, 4)
mask_loc_targets = gt_rpn_loc[mask].reshape(-1, 4)
#print(mask_loc_preds.shape, mask_loc_targets.shape)


#loss start
t = np.abs(mask_loc_targets - mask_loc_preds)
rpn_loc_loss = mxnet.nd.npx.smooth_l1(t).sum()

rpn_lambda = 10.
N_reg = (gt_rpn_score >0).sum()
rpn_loc_loss = rpn_loc_loss.sum() / N_reg
rpn_loss = rpn_cls_loss + (rpn_lambda * rpn_loc_loss)
#loss over

pred_anchor_locs_numpy = pred_anchor_locs[0]
objectness_score_numpy = objectness_score[0]
dy = pred_anchor_locs_numpy[:, 0::4] # dy
dx = pred_anchor_locs_numpy[:, 1::4] # dx
dh = pred_anchor_locs_numpy[:, 2::4] # dh
dw = pred_anchor_locs_numpy[:, 3::4] # dw

# ctr_y = dy predicted by RPN * anchor_h + anchor_cy
# ctr_x similar
# h = exp(dh predicted by RPN) * anchor_h
# w similar
ctr_y = dy * anc_height[:, np.newaxis] + anc_ctr_y[:, np.newaxis]
ctr_x = dx * anc_width[:, np.newaxis] + anc_ctr_x[:, np.newaxis]
h = np.exp(dh) * anc_height[:, np.newaxis]
w = np.exp(dw) * anc_width[:, np.newaxis]

roi = np.zeros(pred_anchor_locs_numpy.shape, dtype=anchor_locs.dtype)
roi[:, 0::4] = ctr_y - 0.5 * h
roi[:, 1::4] = ctr_x - 0.5 * w
roi[:, 2::4] = ctr_y + 0.5 * h
roi[:, 3::4] = ctr_x + 0.5 * w

# clip the predicted boxes to the image
img_size = (800, 800) #Image size
roi[:, slice(0, 4, 2)] = np.clip(roi[:, slice(0, 4, 2)], 0, img_size[0])
roi[:, slice(1, 4, 2)] = np.clip(roi[:, slice(1, 4, 2)], 0, img_size[1])

# Remove predicted boxes with either height or width < threshold.
min_size=16
hs = roi[:, 2] - roi[:, 0]
ws = roi[:, 3] - roi[:, 1]
keep = np.where((hs >= min_size) & (ws >= min_size))[0] #min_size=16

roi = roi[keep, :]
roi_old = roi
score = objectness_score_numpy[keep]
#print(keep.shape, roi.shape, score.shape)

# Sort all (proposal, score) pairs by score from highest to lowest
order = np.ravel(score).argsort()[::-1]
#print(order.shape)

#Take top pre_nms_topN (e.g. 12000 while training and 300 while testing)
order = order[:n_train_pre_nms]
roi = roi[order, :]

# Take all the roi boxes [roi_array]
y1 = roi_old[order, 0]
x1 = roi_old[order, 1]
y2 = roi_old[order, 2]
x2 = roi_old[order, 3]
# Find the areas of all the boxes [roi_area]
areas = (x2 - x1 + 1) * (y2 - y1 + 1)

#order = score.argsort()[::-1]
keep = []
while (order.size > 0):
    i = order[0] #take the 1st elt in order and append to keep 
    keep.append(i)
    xx1 = np.maximum(x1[0], x1[1:]) 
    yy1 = np.maximum(y1[0], y1[1:])
    xx2 = np.minimum(x2[0], x2[1:])
    yy2 = np.minimum(y2[0], y2[1:])
    w = np.maximum(0.0, xx2 - xx1 + 1)
    h = np.maximum(0.0, yy2 - yy1 + 1)
    inter = w * h
    ovr = inter / (areas[0] + areas[1:] - inter)
    inds = np.where(ovr <= nms_thresh)[0] +1
    order = order[inds]
    x1 = x1[inds]
    y1 = y1[inds]
    x2 = x2[inds]
    y2 = y2[inds]
    areas = areas[inds]
keep = keep[:n_train_post_nms] # while training/testing , use accordingly
roi = roi_old[keep] # the final region proposals

ious = box_iou(roi,bbox)    #find iou of roi with gt bbox

# Find out which ground truth has high IoU for each region proposal, Also find the maximum IoU
gt_assignment = ious.argmax(axis=1)
max_iou = ious.max(axis=1)
#print(gt_assignment)
#print(max_iou)

# Assign the labels to each proposal
gt_roi_label = labels[gt_assignment]

# Select the foreground rois as per the pos_iou_thesh and 
# n_sample x pos_ratio (128 x 0.25 = 32) foreground samples.
pos_roi_per_image = 32 
pos_index = np.where(max_iou >= pos_iou_thresh)[0]
pos_roi_per_this_image = int(min(pos_roi_per_image, pos_index.size))
if pos_index.size > 0:
    pos_index = np.random.choice(
        pos_index, size=pos_roi_per_this_image, replace=False)
#print(pos_roi_per_this_image)
#print(pos_index)

# Similarly we do for negitive (background) region proposals
neg_index = np.where((max_iou < neg_iou_thresh_hi) &
                             (max_iou >= neg_iou_thresh_lo))[0]
neg_roi_per_this_image = n_sample - pos_roi_per_this_image
neg_roi_per_this_image = int(min(neg_roi_per_this_image, neg_index.size))
if  neg_index.size > 0 :
    neg_index = np.random.choice(
        neg_index, size=neg_roi_per_this_image, replace=False)
    

# Now we gather positve samples index and negitive samples index, 
# their respective labels and region proposals

keep_index = np.append(pos_index, neg_index)
gt_roi_labels = gt_roi_label[keep_index]
gt_roi_labels[pos_roi_per_this_image:] = 0  # negative labels --> 0
sample_roi = roi[keep_index]
#print(sample_roi.shape)

# Pick the ground truth objects for these sample_roi and 
# later parameterize as we have done while assigning locations to anchor boxes in section 2.
bbox_for_sampled_roi = bbox[gt_assignment[keep_index]]
#print(bbox_for_sampled_roi.shape)

height = sample_roi[:, 2] - sample_roi[:, 0]
width = sample_roi[:, 3] - sample_roi[:, 1]
ctr_y = sample_roi[:, 0] + 0.5 * height
ctr_x = sample_roi[:, 1] + 0.5 * width

base_height = bbox_for_sampled_roi[:, 2] - bbox_for_sampled_roi[:, 0]
base_width = bbox_for_sampled_roi[:, 3] - bbox_for_sampled_roi[:, 1]
base_ctr_y = bbox_for_sampled_roi[:, 0] + 0.5 * base_height
base_ctr_x = bbox_for_sampled_roi[:, 1] + 0.5 * base_width

eps = np.finfo(height.dtype).eps
height = np.maximum(height, eps)
width = np.maximum(width, eps)

dy = (base_ctr_y - ctr_y) / height
dx = (base_ctr_x - ctr_x) / width
dh = np.log(base_height / height)
dw = np.log(base_width / width)
gt_roi_locs = np.vstack((dy, dx, dh, dw)).transpose()



In [5]:
rois = np.array(sample_roi, dtype=np.float32)
roi_indices = np.zeros((len(rois),), dtype=np.float32)

#print(rois.shape, roi_indices.shape)

indices_and_rois = np.concatenate([roi_indices[:, None], rois], axis=1)
xy_indices_and_rois = indices_and_rois[:, [0, 2, 1, 4, 3]]
indices_and_rois = xy_indices_and_rois
#print(xy_indices_and_rois.shape)

#roi pooling 
size = (7, 7)
output = npx.roi_pooling(out_map,xy_indices_and_rois, size,1.0/16.0)
kk = output.reshape(output.shape[0],-1)

kk = roi_head_classifier(kk)
roi_cls_loc = cls_loc(kk)
roi_cls_score = score_l(kk)

gt_roi_loc = np.array(gt_roi_locs)
gt_roi_label = np.array(gt_roi_labels, dtype = np.float32)

#loss2_1 start
##roi_cls_loss = cross_entropy(roi_cls_score, gt_roi_label)
ce_loss = gluon.loss.SoftmaxCrossEntropyLoss()
roi_cls_loss= ce_loss(roi_cls_score, gt_roi_label).mean()
#loss2_1 over

#regression ( loss2_2) start
n_sample = roi_cls_loc.shape[0]
roi_loc = roi_cls_loc.reshape(n_sample, -1,4)
roi_loc = roi_loc[np.arange(0,n_sample),gt_roi_label]
pos = (gt_roi_label > 0).reshape(-1,1)
mask = np.ones((pos.shape[1],4), dtype= pos.dtype)
mask = pos * mask
mask_loc_preds = roi_loc[mask].reshape(-1, 4)
mask_loc_targets = gt_roi_loc[mask].reshape(-1, 4)
x = np.abs(mask_loc_targets - mask_loc_preds)
roi_loc_loss = mxnet.nd.npx.smooth_l1(x).sum()

#loss2_2 over
# loss2 sum start
roi_lambda = 10.
roi_loss = roi_cls_loss + (roi_lambda * roi_loc_loss)
#loss2 sum over




In [6]:
#total loss 
total_loss = rpn_loss + roi_loss


In [7]:
total_loss

array(57.394974)